In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, Subset

# import numpy as np
# from qiskit import QuantumCircuit, transpile
# from qiskit_aer import AerSimulator

from layers.WHT import WHTConv2D

In [9]:
# download format
# turns MNIST images to PyTorch tensors and normalizes between [-1,1] centered at 0
transform = transforms.Compose([
    transforms.RandomCrop(28, padding=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# download data to computer
raw_train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

raw_test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,            # testing data so labels are unknown during training
    download=True,
    transform=transform
)

In [10]:
# apply ht to all images before training so that i dont run qiskit every epoch
# ht qiskit class
# ht = HadamardTransform()

# def precompute(dataset):

#     temp_image_list, temp_label_list = [], []

#     for image, label in DataLoader(dataset, batch_size=1, shuffle=False):
#         image_ht = ht(image[0])                         # get rid of batch dimensions so i can apply the ht
#         temp_image_list.append(image_ht.unsqueeze(0))   # undo previous step and append to temp_image_list
#         temp_label_list.append(label)

#     concatinated_image_dataset = torch.cat(temp_image_list, dim=0) # combines all tensors in temp lists to a single tensor with batch dimensions
#     concatinated_label_dataset = torch.cat(temp_label_list, dim=0)
#     return TensorDataset(concatinated_image_dataset, concatinated_label_dataset)

# sanity test lines to make sure the cnn works
# it works but it takes a super long time to run all images in the fashion mnist
raw_train_subset = Subset(raw_train_dataset, range(10000))
raw_test_subset = Subset(raw_test_dataset, range(1000))

# use raw_train_small and raw_test_small to decrease number of training and test images
# use raw_train_dataset and raw_test_dataset for entire dataset
train_dataset = raw_train_subset
test_dataset  = raw_test_subset

In [11]:
# loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=128,         # each epoch is 128 samples
    shuffle=True            # randomize after each training epoch
)

test_loader = DataLoader(
    test_dataset,
    batch_size=256,         # each epoch is 256 samples
    shuffle=False
)

# test shapes of pytorch datasets
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)

# print("train size", len(train_loader.dataset))
# print("test size", len(test_loader.dataset))

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [12]:
# CNN model
class CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.wht = WHTConv2D(height=28, width=28, in_channels=1, out_channels=1, pods=1, residual=True)

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.drop = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.wht(x)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, start_dim=1)
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# create model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.003, weight_decay=0.01)

In [13]:
# training loop
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/15], Loss: 1.4130
Epoch [2/15], Loss: 1.0382
Epoch [3/15], Loss: 0.9674
Epoch [4/15], Loss: 0.9445
Epoch [5/15], Loss: 0.9085
Epoch [6/15], Loss: 0.8977
Epoch [7/15], Loss: 0.8810
Epoch [8/15], Loss: 0.8661
Epoch [9/15], Loss: 0.8569
Epoch [10/15], Loss: 0.8601
Epoch [11/15], Loss: 0.8326
Epoch [12/15], Loss: 0.8200
Epoch [13/15], Loss: 0.8137
Epoch [14/15], Loss: 0.8124
Epoch [15/15], Loss: 0.8023


In [14]:
# get accuracy
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 89.00%
